<a href="https://colab.research.google.com/github/JimKing100/NFL-Live/blob/master/data-science/actuals/Actuals_Rookie_Kickers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import pandas as pd

In [0]:
# Load the raw data
rookie_kicker_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/rookies_non_kicker.csv')
kicking_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/kicking.csv')
missed_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/missedfg.csv')
conv_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/conv.csv')

In [3]:
# The dataframe of all 2019 non-rookie kickers and their rookie year
rookie_kicker_df.head()

,player,first,last,name,position1,start
0,AV-0400,Adam,Vinatieri,Adam Vinatieri,K,1996
1,MB-4600,Matt,Bryant,Matt Bryant,K,2002
2,RG-1500,Robbie,Gould,Robbie Gould,K,2005
3,MN-0800,Mike,Nugent,Mike Nugent,K,2005
4,SG-0800,Stephen,Gostkowski,Stephen Gostkowski,K,2006


In [4]:
# The kicking dataframe - raw kicking data for all 2019 kickers for seasons 2000-2019
kicking_df.head()

,uid,gid,player,del,first,last,name,position1,pat,fgs,fgm,fgl,fp,game,seas,year,team
0,15,8,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,1,1,0,0,4,1,5,2000,NE
1,59,30,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,1,4,0,0,13,2,5,2000,NE
2,76,38,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,1,0,0,0,1,3,5,2000,NE
3,100,51,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,0,0,1,0,4,4,5,2000,NE
4,122,63,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,4,0,0,0,4,5,5,2000,NE


In [5]:
# The missed field goal dataframe - raw missed field goal data for all 2019 kickers for seasons 2000-2019
missed_df.head()

,pid,gid,year,week,fgxp,fkicker,first,last,name,dist,good
0,517,4,2000,1,FG,JH-0500,John,Hall,John Hall,52,0
1,583,4,2000,1,FG,JH-0500,John,Hall,John Hall,48,0
2,877,6,2000,1,FG,KH-1500,Kris,Heppner,Kris Heppner,52,0
3,1203,8,2000,1,FG,AV-0400,Adam,Vinatieri,Adam Vinatieri,55,0
4,1435,9,2000,1,FG,DB-3900,Doug,Brien,Doug Brien,52,0


In [6]:
conv_df.head()

,pid,gid,year,week,type,bc,first_bc,last_bc,name_bc,psr,first_psr,last_psr,name_psr,trg,first_trg,last_trg,name_trg,conv
0,456,3,2000,1,PASS,NaN,NaN,NaN,NaN,RC-2800,Randall,Cunningham,Randall Cunningham,RI-0200,Raghib,Ismail,Raghib Ismail,1
1,1408,9,2000,1,RUSH,JS-6500,James,Stewart,James Stewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2528,16,2000,2,PASS,NaN,NaN,NaN,NaN,TB-0200,Tony,Banks,Tony Banks,BC-1600,Ben,Coates,Ben Coates,1
3,2779,17,2000,2,PASS,NaN,NaN,NaN,NaN,BF-0100,Brett,Favre,Brett Favre,TD-0800,Tyrone,Davis,Tyrone Davis,1
4,3052,19,2000,2,RUSH,MA-1000,Mike,Anderson,Mike Anderson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [0]:
# Pull the stats to calculate the kicker points - the fp column is the points total for each game
def pull_stats(df, df1, df2, df3, col, p, y, g):
    missedfg_points = 0
    conversion_points = 0

    # Fill the empty values with NaN prior to start year, else use zero
    if (df[col].loc[(df['player']==p) & (df['year']==y) & (df['game']==g)].empty):
        start_year = df1['start'].loc[(df1['player']==p)].iloc[0]
        if y < start_year:
            points = float('NaN')
        else:
            points = 0
    else:
        conversion_points1 = df3['conv'].loc[(df3['bc']==p) & (df3['year']==y) & (df3['week']==g)].count()
        conversion_points2 = df3['conv'].loc[(df3['psr']==p) & (df3['year']==y) & (df3['week']==g)].count()
        conversion_points3 = df3['conv'].loc[(df3['trg']==p) & (df3['year']==y) & (df3['week']==g)].count()

        conversion_points = conversion_points1 + conversion_points2 + conversion_points3
        missedfg_points = df2['good'].loc[(df2['fkicker']==p) & (df2['year']==y) & (df2['week']==g)].count()
        points = df[col].loc[(df['player']==p) & (df['year']==y) & (df['game']==g)].iloc[0]

    return points - missedfg_points + conversion_points


In [0]:
# Add a row to the final_df dataframe
# Each row represents the actual kicking points for each game for each kicker
def add_row(df, p, f, l, n, pos, value_list):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position1': pos,
                    'game1': value_list[0], 
                    'game2': value_list[1],
                    'game3': value_list[2], 
                    'game4': value_list[3], 
                    'game5': value_list[4], 
                    'game6': value_list[5], 
                    'game7': value_list[6], 
                    'game8': value_list[7], 
                    'game9': value_list[8],
                    'game10': value_list[9],
                    'game11': value_list[10],
                    'game12': value_list[11],
                    'game13': value_list[12],
                    'game14': value_list[13],
                    'game15': value_list[14],
                    'game16': value_list[15]
                    }, ignore_index=True)
    
    return df

In [0]:
# The main code for iterating through the kicker list, calculating the points and adding the rows
# to the final_df dataframe.
column_names = ['player',
                'first',
                'last',
                'name',
                'position1', 
                'game1',
                'game2',
                'game3', 
                'game4', 
                'game5', 
                'game6', 
                'game7', 
                'game8', 
                'game9',
                'game10',
                'game11',
                'game12',
                'game13',
                'game14',
                'game15',
                'game16'
                ]

player_list = rookie_kicker_df['player'].tolist()

final_df = pd.DataFrame(columns = column_names)

for player in player_list:

    first = rookie_kicker_df['first'].loc[(rookie_kicker_df['player']==player)].iloc[0]
    last = rookie_kicker_df['last'].loc[(rookie_kicker_df['player']==player)].iloc[0]
    name = rookie_kicker_df['name'].loc[(rookie_kicker_df['player']==player)].iloc[0]
    position1 = rookie_kicker_df['position1'].loc[(rookie_kicker_df['player']==player)].iloc[0]
    year = rookie_kicker_df['start'].loc[(rookie_kicker_df['player']==player)].iloc[0]

    player_scores = []

    for game in range(1,17):
        score = pull_stats(kicking_df, rookie_kicker_df, missed_df, conv_df, 'fp', player, year, game)

        player_scores.append(score)

    final_df = add_row(final_df, player, first, last, name, position1, player_scores)

In [10]:
final_df.head(50)

,player,first,last,name,position1,game1,game2,game3,game4,game5,game6,game7,game8,game9,game10,game11,game12,game13,game14,game15,game16
0,AV-0400,Adam,Vinatieri,Adam Vinatieri,K,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,MB-4600,Matt,Bryant,Matt Bryant,K,7,14,10,1,3,4,3,6,-1,13,2,10,10,12,8,4
2,RG-1500,Robbie,Gould,Robbie Gould,K,5,4,4,7,7,5,6,7,13,2,10,7,3,3,0,0
3,MN-0800,Mike,Nugent,Mike Nugent,K,0,6,8,3,0,6,2,8,2,15,3,13,10,3,13,0
4,SG-0800,Stephen,Gostkowski,Stephen Gostkowski,K,5,5,0,7,8,4,8,9,5,5,7,8,16,7,10,13
5,MP-2100,Matt,Prater,Matt Prater,K,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,MC-3000,Mason,Crosby,Mason Crosby,K,13,4,7,12,8,3,7,17,9,8,11,12,9,17,1,10
7,NF-0300,Nick,Folk,Nick Folk,K,9,14,10,5,16,9,6,8,8,4,10,15,4,5,9,6
8,SH-0400,Stephen,Hauschka,Stephen Hauschka,K,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,RS-3400,Ryan,Succop,Ryan Succop,K,8,4,2,4,8,14,0,8,11,8,2,8,4,9,4,15


In [0]:
# Save the results to .csv file
final_df.to_csv('/content/actuals_rookie_kickers.csv', index=False)